# Extract, Transform and Load Spotify Data Set
<hr>

### Dependencies

In [2]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import requests
import pymongo
import json

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Extract Data Set

In [2]:
path = "../Resources/Spotify_Daily_Streaming.csv"
spotify_data = pd.read_csv(path)

In [3]:
spotify_data.head(5)

,Date,Track URL,Position,Track Name,Artist,Streams,Country
0,7/20/2017,https://open.spotify.com/track/3RXkboS74UYzN14xTqzPyY,1,NaN,NaN,3457,Lithuania
1,7/31/2018,https://open.spotify.com/track/77wz2VtAwxAwYOGTJrZBKT,1,"Miljonair (feat. SBMG, Lil' Kleine, Boef & Ronnie Flex)",$hirak,238729,Netherlands
2,8/2/2018,https://open.spotify.com/track/77wz2VtAwxAwYOGTJrZBKT,1,"Miljonair (feat. SBMG, Lil' Kleine, Boef & Ronnie Flex)",$hirak,245639,Netherlands
3,7/23/2017,https://open.spotify.com/track/6mw02h3qbWmuq2bJCRjRAe,1,Millionär,187 Strassenbande,387632,Germany
4,6/29/2018,https://open.spotify.com/track/2iUXsYOEPhVqEBwsqP70rE,1,Youngblood,5 Seconds of Summer,261795,Australia


### Transform 

In [4]:
print("We have a total of " + str(len(spotify_data)) + " rows in the data set")

We have a total of 8469130 rows in the data set


In [5]:
spotify_data.dtypes

Date          object
Track URL     object
Position       int64
Track Name    object
Artist        object
Streams        int64
Country       object
dtype: object

### Change date type from obj to date

In [6]:
spotify_data["Date"] = pd.to_datetime(spotify_data["Date"], format="%m/%d/%Y")

In [7]:
spotify_data.dtypes

Date          datetime64[ns]
Track URL             object
Position               int64
Track Name            object
Artist                object
Streams                int64
Country               object
dtype: object

### Verify NAs and kill them all

In [8]:
spotify_data["Artist"].isna().value_counts()

False    8466361
True        2769
Name: Artist, dtype: int64

In [9]:
spotify_data_wNA = spotify_data.dropna()
spotify_data_wNA["Artist"].isna().value_counts()

False    8466353
Name: Artist, dtype: int64

### Separate Global and Country Data

In [10]:
spotify_data_global = spotify_data_wNA[spotify_data_wNA["Country"]=="Global"] #Global Data Set
spotify_data_bycountry = spotify_data_wNA[spotify_data_wNA["Country"]!="Global"] #Country Data Set

### Get unique Artist List to retrieve info (Genre, Followers and Image) from Spotify API

In [11]:
# Get unique Artists from all the Data Set
spotify_artists = spotify_data_wNA.groupby("Artist").sum().sort_values(by=['Streams'],ascending=False).reset_index()
len(spotify_artists["Artist"])

11046

In [15]:
# spotify_artists = spotify_artists[~spotify_artists.Artist.isin(spotify_artists_df1["Artist"])]
# spotify_artists

### Load data into MongoDB Collection adding Spotify Genre by Artist

In [ ]:
token="BQAqFieWd9Vs7_PadOTnr5aa5Z718R_mmO6WACUnALMgjtzORXuE1eV6wxBonSkIlbKt924WCoIdF2Ia9QSgFlKzw2VONVEbgzTYTMUobMQTP4CSybOuLgCyAc8dasFXh8ycBwfmtLKvrHb5D9v7c3t_DxahFixpPZd-TsBPoCbRgw"

headers_dict = {"Authorization": f'Bearer {token}'}

# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.Spotifydb
collection = db.SpotifyArtists
# collection.drop()

rows=len(spotify_artists)
artist_dict={}
# spotify_artist_dict=[]

for row in range(0,rows):
    #Hacer el llamado a la API de Spotify usando spotify_data_wNA.iloc[row,4]
    artist = spotify_artists.iloc[row,0]
    if (artist[0]=="#"):
        artist=artist[1:]
    
    response = requests.get(f'https://api.spotify.com/v1/search?q={artist}&type=artist', headers=headers_dict).json()
#     print(response)
    artists = response["artists"]
    
    try:
        items = artists["items"][0]
        genres = items["genres"]
        genre = items["genres"][0]
        followers = items["followers"]["total"]
        image = items["images"][0]["url"]
        
    except:
#         print(artists)
        genre = 'None'
        genres = ['Unavailable']
        followers = 'None'
        image = 'No Image Found'
        
        
    artist_dict={'Artist':artist,
                 'Genre':genre,
                 'Genres': genres,
                 'Followers': followers,
                 'Image': image
                 }
    collection.insert_one(artist_dict)
#     spotify_artist_dict.append(artist_dict)
   

In [30]:
# pd.read_json()
spotify_artists_json = json.load(open("../Resources/SpotifyArtists.json"))
spotify_artists_df = pd.DataFrame(spotify_artists_json)
spotify_artists_df[spotify_artists_df["Artist"]=="TocoParaVos"]

,Artist,Followers,Genre,Genres,Image
3014,TocoParaVos,251948,cumbia pop,[cumbia pop],https://i.scdn.co/image/ab6761610000e5eb74053255cb8a8eac221c760d


In [13]:
spotify_artists_df.loc[3014,"Artist"] = "#TocoParaVos"
spotify_artists_df[spotify_artists_df["Artist"]=="#TocoParaVos"]

,Artist,Followers,Genre,Genres,Image
3014,#TocoParaVos,251948,cumbia pop,[cumbia pop],https://i.scdn.co/image/ab6761610000e5eb74053255cb8a8eac221c760d


### Complete Final Data Base for Historial Top 200 with Genre and Image 

In [14]:
# Complete Data set by country with genre
spotify_data_complete_country = pd.merge(spotify_data_bycountry, spotify_artists_df, how="left", on="Artist")
spotify_data_complete_country = spotify_data_complete_country[["Date", "Track Name", "Artist","Genre","Streams","Country","Position","Genres","Track URL","Image"]]

# Complete Data set Global with genre
spotify_data_complete_global = pd.merge(spotify_data_global, spotify_artists_df, how="left", on="Artist")
spotify_data_complete_global = spotify_data_complete_global[["Date", "Track Name", "Artist","Genre","Streams","Country","Position","Genres","Track URL","Image"]]

### Added Column Month & Year to the data set

In [23]:
# Complete Data set by country with MonthYear
# spotify_data_complete_country["MonthYear"] = (pd.DatetimeIndex(spotify_data_complete_country['Date']).year).astype(str)+"-"+(spotify_data_complete_country['Date'].dt.strftime('%m')).astype(str)+"-01"
spotify_data_complete_country["MonthYear"] = (spotify_data_complete_country['Date'].dt.strftime('%m')).astype(str)+"-01-"+(pd.DatetimeIndex(spotify_data_complete_country['Date']).year).astype(str)
spotify_data_complete_country["MonthYear"] = pd.to_datetime(spotify_data_complete_country["MonthYear"], format="%m-%d-%Y")


# Complete Data set Global with MonthYear
spotify_data_complete_global["MonthYear"] = (spotify_data_complete_global['Date'].dt.strftime('%m')).astype(str)+"-01-"+(pd.DatetimeIndex(spotify_data_complete_global['Date']).year).astype(str)
spotify_data_complete_global["MonthYear"] = pd.to_datetime(spotify_data_complete_global["MonthYear"], format="%m-%d-%Y")

In [12]:
# spotify_data_complete_country.head()

## Load Final Spotify Data Base to MongoDB

### Load Top 200 by Country

In [26]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.Spotifydb
collection = db.Top200byCountry
collection.drop()


rows=len(spotify_data_complete_country)
data=spotify_data_complete_country
songs_dict={}
x=0


for row in range(0,10000):
        
    songs_dict={'Date':data.iloc[row,0],
                'MonthYear':data.iloc[row,10],
                'Track_Name':data.iloc[row,1],
                'Artist':data.iloc[row,2],
                'Genre':data.iloc[row,3],
                'Streams':data.iloc[row,4].astype(float),
                'Country':data.iloc[row,5],
                'Position':data.iloc[row,6].astype(float),
                'Genres':data.iloc[row,7],
                'Track_URL':data.iloc[row,8],
                'Image':data.iloc[row,9],
                 }
    
    collection.insert_one(songs_dict)

### Songs popularity

In [11]:
TopGlobal_json = pd.read_json('../Resources/TopGlobal.json', encoding='UTF-8')
TopGlobal_df = pd.DataFrame(TopGlobal_json)
# TopGlobal_df.head()

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.Spotifydb
collection = db.TopGlobal2
collection.drop()


rows = len(TopGlobal_df)
data = TopGlobal_df
songs_dict = {}
x=0


for row in range(0,rows):
        
    songs_dict={'Artist Name':data.iloc[row,1],
                'Track Name':data.iloc[row,2],
                'Danceability':data.iloc[row,3].astype(float),
                'Energy':data.iloc[row,4].astype(float),
                'Key':data.iloc[row,5].astype(float),
                'Loudness':data.iloc[row,6].astype(float),
                'Acousticness':data.iloc[row,7].astype(float),
                'Instrumentalness':data.iloc[row,8].astype(float),
                'Liveness':data.iloc[row,9].astype(float),
                'Valence':data.iloc[row,10].astype(float),
                'Tempo':data.iloc[row,11].astype(float),
                'Track Reference':data.iloc[row,12],
                 }
    
    collection.insert_one(songs_dict)